+# Módulo 7: Análisis filogenético


¡Bienvenido al curso! Hay algunas instrucciones muy importantes que debe seguir si desea guardar una copia de este módulo en su cuenta de Google Drive:

1.) Haga clic en "Archivo" en la esquina superior izquierda y seleccione "Guardar una copia en Drive"


2.) Haga clic en el nombre del notebook en la esquina superior izquierda y reemplace por el nombre que desee.

**Sus cambios no se guardarán si no realiza este paso.**

Antes de iniciar este módulo le recomendamos ver los videos de esta playlist: https://www.youtube.com/watch?v=BcXr7QhJ5AE&list=PLfovZnX0TvKu_wFgoZSN0DuKQ-sarlmg3&index=2

### Instalemos los paquetes de análisis necesarios.

Para esta parte usaremos: 
**seqtk** para evaluar la calidad del genoma (https://github.com/lh3/seqtk)  
**Mafft** para la alineación del genoma (https://mafft.cbrc.jp/alignment/software/)  
**snp-site** Para una evaluación rápida de de diferencia de SNP (https://github.com/sanger-pathogens/snp-sites)  
**Fasttree** para la construcción de árboles filogenéticos (http://www.microbesonline.org/fasttree/)  
**Phylo** d para visualización rápida de árboles con biophyton (https://biopython.org/wiki/Phylo). 

*Nota*: Hay muchos programas de Visualización de árboles. Los más utilizados comúnmente son ggtree (R), ete3 (Python) e itol (https://itol.embl.de/).  

Cuenta microreact (https://microreact.org/), donde puede iniciar sesión con su cuenta de Google.

In [ ]:
!python --version

Python 3.7.15


In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

In [ ]:
!conda install -c conda-forge biopython

In [ ]:
!conda install -c bioconda mafft snp-sites fasttree seqtk snp-dists


### Descargue los datasets para ser analizados
 
En este módulo utilizaremos los datos analizados en el módulo 3. Estos datos pertenecen al artículo "[Overview of the SARS-CoV-2 genotypes circulating in Latin America during 2021]( https://doi.org/10.3389/fpubh.2023.1095202 )" publicado por Molina-Mora et al. 2023, como parte de un proyecto de CABANA.

In [ ]:
!wget https://zenodo.org/records/10888461/files/module_4.tar

In [ ]:
!tar -xvf module_4.tar

In [ ]:
%cd module_4/

Primero evaluaremos la calidad de los genomas para determinar si son suficientemente buenos para el análisis. Para SARS-CoV-2, estamos principalmente interesados en el número de Ns. El paquete que utilizaremos es seqtk comp, que ofrece muchas estadísticas para archivos en formato fasta.

Output format: chr, length, #A, #C, #G, #T, #2, #3, #4, #CpG, #tv, #ts, #CpG-ts

9th column, #4 is the number of Ns, 4 ambiguities, ie A, G, T, or C.

Si ha separado los archivos FASTA, podría ser más fácil combinar los archivos. De lo contrario, debe analizar por los archivos del genoma por separado.

In [ ]:
#seqtk comp nos permite obtener la calidad de un archivo fasta
#cut -f 1,9 Selecciona la columna 1 y 9, la información que necesitamos.
!seqtk comp gisaid_hcov-19_2024_03_27_02.fasta    | cut -f 1,9 > gisaid_hcov-19_quality.tsv

In [ ]:
!cat gisaid_hcov-19_quality.tsv

La calidad se ve bien, sin embargo, las muestras EPI_ISL_10072006 y EPI_ISL_7961355  no son tam buenas como las demás. Tienen 1139 y 1628 bases ambiguas, pero son lo suficientemente buenas para el análisis. En general, requerimos el 90% o menos de **3000** Ns en el genoma para ensamblar y realizar el análisis de árboles filogenéticos. Tener el 70% del genoma ensamblado es el valor predeterminado en Pangolin para la asignación de linaje.

### Alineamiento

Utilizaremos mafft para la alineación. Es bastante rápido y bastante preciso.

Existen muchas opciones para alinear secuencias en mafft. `--auto` es una buena opción donde el programa elige automáticamente el algoritmo más eficiente (un buen equilibrio entre velocidad y precisión). La alineación podría llevar días si se utilizara el algoritmo más preciso para alinear secuencias largas. Si está alineando secuencias cortas, como una región del gen/proteína de la espiga, puede utilizar opciones más precisas. El sitio web de mafft tiene buenos ejemplos de qué usar en cada caso.

Para que mafft alinee las secuencias, en nuestro caso genomas completos, necesitas combinar las secuencias que deseas alinear en un solo archivo. Lo bueno es que ya has hecho eso y como todos los genomas son buenos, no necesitamos excluir ninguno de la alineación.

In [ ]:
!mafft --auto gisaid_hcov-19_2024_03_27_02.fasta > gisaid_hcov_aligned.fasta

### Mirada rápida a las diferencias de los genomas
snp-site es un software realmente bueno para darte una idea de cuán relacionados están tus genomas de interés. Para conjuntos de datos pequeños como este, es realmente útil. Analiza tu alineación y te proporciona un alineamiento de SNP de manera predeterminada. También puede proporcionarte información de SNP en formato VCF.

In [ ]:
!snp-sites gisaid_hcov_aligned.fasta

In [ ]:
#puede mirar la distancia de SNP entre pares de muestras usando snp-dist
!snp-dists gisaid_hcov_aligned.fasta

La alineación es un poco más complicada con más diferencias, por lo tanto, necesitamos un árbol filogenético para visualizar las relaciones

In [ ]:
!fasttree -nt -gtr -gamma gisaid_hcov_aligned.fasta > gisaid_hcov_aligned.nwk

In [ ]:
!cat gisaid_hcov_aligned.nwk
#Como puede ver, es bastante difícil interpretar este formato sin visualizar en forma de árbol

In [ ]:
from Bio import Phylo

tree = Phylo.read("gisaid_hcov_aligned.nwk", "newick")
print("tree")
Phylo.draw_ascii(tree)

In [ ]:
# Ahora vamos a agregar el genoma de referencia 2020
!mafft --auto --add gisaid_hcov_aligned.fasta reference_genome.fasta > gisaid_hcov_aligned_ref.fasta

In [ ]:
!fasttree -nt -gtr -gamma gisaid_hcov_aligned_ref.fasta > gisaid_hcov_aligned_ref.nwk


Eche un vistazo a la alineación de SNP también usando snp-site. Verá que el uso de la referencia permitió la identificación de bases ancestrales, por lo que ahora sabemos qué cambios en los nucleótidos son mutaciones del tipo salvaje. Anteriormente, el algoritmo de árbol suponía que la base mayoritaria es la secuencia ancestral, que no es correcta.

In [ ]:
from Bio import Phylo

treeOutgroup = Phylo.read("gisaid_hcov_aligned_ref.nwk", "newick")
treeOutgroup.root_with_outgroup({"name": "nCoV2019|Wuhan-Hu-1|MN908947|China|Wuhan|2019-12"}) 
print("treeOutgroup")
Phylo.draw_ascii(treeOutgroup)

### Observar la demostración/video de microreact y nextstrain

Mirar los árboles de esta manera es bastante difícil, existen algunas buenas herramientas que permiten árboles interactivos y visualización de metadatos.

https://docs.microreact.org/   
https://nextstrain.org/community/narratives/ESR-NZ/GenomicsNarrativeSARSCoV2/aotearoa-border-incursions

### Otras herramientas para conjuntos de datos y construcción de árboles más grandes
mafft incluso en el modelo de precisión mínima puede ser bastante lento una vez que tenga 1000 de genomas para alinearse. Para acelerar las cosas, puede agregar nuevos genomas a las alineaciones existentes o dividir los genomas en grupos como linajes o sublinajes y luego fusionar las alineaciones. El sitio web de mafft tiene una buena explicación de estos trucos y sus problemas.

nextalign es otra herramienta que puede usar.¡Es súper rápida!(https://docs.nextstrain.org/projects/nextclade/en/stable/user/nextalign-cli.html)

iqtree2 es un paquete integral (no baysiano) de construcción de filogenia que puede probar los mejores modelos de substitución  para su conjunto de datos, realizar análisis bootstrap, comparar filogenia, etc.  Es probablemente una de las mejores herramientas de construcción de árbol.(http://www.iqtree.org/)

usher es un paquete de software que permite agregar secuencias a los árboles existentes sin volver a hacer toda la alineación.(https://www.nature.com/articles/s41588-021-00862-7) Usamos esto cuando analizamos el conjunto de datos de vuelo. Hay algunas advertencias y problemas que debe tener en cuenta. Si la alineación existente está realmente limpia y las secuencias a agregar son de alta calidad, usar usher es una buena forma rápida de obtener algunos resultados preliminares rápidos. Sin embargo, no trabaja muy bien con Ns o gaps.

Para la filogenia bayesiana temporal , a menudo se usa BEAST2. (https://www.beast2.org/) https://www.nature.com/articles/s41559-017-0280-x

## Tarea

> **Hay un clado que contiene las 3 secuencias de un solo país. ¿Cuál país es?** 

> **¿Cuál es la distancia entre EPI_ISL_14434222 y EPI_ISL_14434358**

> **¿Cómo lo ve reflejado en el árbol?**

*Adaptado de:*

- SARS-CoV-2 Bioinformatics for Beginners Course - Wellcome Connecting Science
- Viral Genomics and Bioinformatics Asia 2022 - Wellcome Connecting Science


*Modificado por Luisa Sacristán (Universidad de los Andes-CABANA)*